In [ ]:
import os
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point, MultiPoint, box
from shapely.ops import voronoi_diagram, unary_union
import momepy
from tqdm import tqdm as tqdm_base
import random
import warnings
from pathlib import Path
import cartopy.crs as ccrs
from cartopy.io.img_tiles import MapboxTiles
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Suppress warnings
warnings.filterwarnings('ignore')

#########################################################
# CONFIGURATION - ADJUSTED PARAMETERS TO MATCH OLD SCRIPT
#########################################################

# Input Paths
BUILDINGS_PATH = "/home/ls/sites/re-blocking/data/shapefiles/ny-manhattan-buildings/geo_export_a80ea1a2-e8e0-4ffd-862c-1199433ac303.shp"
PARCELS_PATH = "/home/ls/sites/re-blocking/data/shapefiles/ny-manhattan-parcels/NYC_2021_Tax_Parcels_SHP_2203/Kings_2021_Tax_Parcels_SHP_2203.shp"

# Output Settings
OUTPUT_DIR = "brooklyn_comparison"
VORONOI_DIR = os.path.join(OUTPUT_DIR, "voronoi")
SAMPLE_FILE = os.path.join(OUTPUT_DIR, "brooklyn_samples.npy")

# Processing Parameters
BUFFER_DISTANCE = 200
RANDOM_SEED = 42
USE_CITYWIDE_VORONOI = False  # Set to True to use citywide tessellation

# Visualization Settings - MODIFIED FOR BETTER ZOOM LEVEL
FIGURE_SIZE = (7, 7)
DPI = 96
SATELLITE_ZOOM = 18
ZOOM_ADJUSTMENT = 0  # Removed zoom adjustment to match old script
EXTENT_SCALE_FACTOR = 0.7  # Adjusted from 0.9 to find better balance

# Style Parameters - Matching original script
INCLUDE_EDGES = False  # Set to False to remove edge lines around parcels

#########################################################
# FUNCTIONS
#########################################################

def random_hex_color(seed=None):
    """Generate a random hex color, optionally with a seed for reproducibility."""
    if seed is not None:
        random.seed(seed)
    r = random.randint(0, 255)
    g = random.randint(0, 255)
    b = random.randint(0, 255)
    return "#{:02x}{:02x}{:02x}".format(r, g, b)

def manual_voronoi_tessellation(buildings_gdf, boundary_geometry):
    """Manual Voronoi tessellation if momepy fails."""
    if len(buildings_gdf) == 0:
        return gpd.GeoDataFrame(geometry=[], crs=buildings_gdf.crs)
    
    try:
        # Create centroids for all buildings
        centroids = [Point(geom.centroid) for geom in buildings_gdf.geometry]
        
        # Create a multipoint from all centroids
        multipoint = MultiPoint(centroids)
        
        # Generate Voronoi diagram
        voronoi_polys = voronoi_diagram(multipoint, envelope=boundary_geometry)
        
        # Extract polygons and clip with boundary
        clipped_polys = []
        for poly in voronoi_polys.geoms:
            try:
                clipped_poly = poly.intersection(boundary_geometry)
                if clipped_poly.area > 0:
                    clipped_polys.append(clipped_poly)
            except Exception:
                continue
        
        # Create GeoDataFrame
        voronoi_gdf = gpd.GeoDataFrame(geometry=clipped_polys, crs=buildings_gdf.crs)
        
        # Add colors
        voronoi_gdf['color'] = [random_hex_color(i) for i in range(len(voronoi_gdf))]
        
        return voronoi_gdf
    
    except Exception as e:
        print(f"Manual Voronoi also failed: {e}")
        return gpd.GeoDataFrame(geometry=[], crs=buildings_gdf.crs)

def generate_voronoi_tessellation(buildings_gdf, boundary_geometry):
    """Generate Voronoi tessellation based on building centroids."""
    # Skip momepy and just use the manual method directly
    # This avoids the 'eID' error you're encountering
    return manual_voronoi_tessellation(buildings_gdf, boundary_geometry)

def render_voronoi_tessellation(voronoi_gdf, area_geometry, output_path, buildings_gdf=None, include_buildings=False):
    """
    Render Voronoi tessellation on satellite imagery with optional building footprints.
    
    Args:
        voronoi_gdf: GeoDataFrame with Voronoi cells
        area_geometry: Shapely geometry of the area
        output_path: Path to save the rendered image
        buildings_gdf: GeoDataFrame with buildings (optional)
        include_buildings: Whether to include building footprints
    """
    # Get Mapbox token from environment
    mapbox_token = os.environ.get('MAPBOX_ACCESS_TOKEN')
    if not mapbox_token:
        print("Warning: MAPBOX_ACCESS_TOKEN not found in environment variables")
        return
    
    # Create figure
    fig = plt.figure(figsize=FIGURE_SIZE, dpi=DPI)
    
    # Use Mapbox satellite imagery
    tiler = MapboxTiles(mapbox_token, 'satellite-v9')
    ax = fig.add_subplot(1, 1, 1, projection=tiler.crs)
    
    # Set extent based on area geometry
    bounds = area_geometry.bounds
    
    # Calculate the centroid and max distance for square aspect ratio
    dist1 = bounds[2] - bounds[0]
    dist2 = bounds[3] - bounds[1]
    max_dist = max(dist1, dist2) / 2
    
    # Apply scaling factor to adjust the extent
    scaled_max_dist = max_dist * EXTENT_SCALE_FACTOR
    
    centroid_x = (bounds[2] + bounds[0]) / 2
    centroid_y = (bounds[3] + bounds[1]) / 2
    
    # Set extent with square aspect ratio
    ax.set_extent([
        centroid_x - scaled_max_dist,
        centroid_x + scaled_max_dist,
        centroid_y - scaled_max_dist,
        centroid_y + scaled_max_dist
    ], crs=ccrs.epsg('3857'))
    
    # Add satellite imagery at specified zoom level (no adjustment)
    ax.add_image(tiler, SATELLITE_ZOOM)
    
    # Add Voronoi cells - matching original script's styling without edges
    for idx, row in voronoi_gdf.iterrows():
        if INCLUDE_EDGES:
            ax.add_geometries(
                [row.geometry], 
                crs=ccrs.epsg('3857'),
                facecolor=row['color'],
                edgecolor='white',
                linewidth=0.5,
                alpha=1.0
            )
        else:
            # Original script style - no edges specified
            ax.add_geometries(
                [row.geometry], 
                crs=ccrs.epsg('3857'),
                facecolor=row['color']
            )
    
    # Add building footprints if requested
    if include_buildings and buildings_gdf is not None:
        for idx, row in buildings_gdf.iterrows():
            ax.add_geometries(
                [row.geometry], 
                crs=ccrs.epsg('3857'),
                facecolor='black',
                edgecolor='white' if INCLUDE_EDGES else None,
                linewidth=0.3 if INCLUDE_EDGES else 0,
                alpha=0.8
            )
    
    # Remove axes
    ax.set_axis_off()
    
    # Save figure with tight layout
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0, dpi=DPI)
    plt.close(fig)

def create_view_box(center_point, width, height):
    """Create a rectangular view box around a center point."""
    return box(
        center_point.x - width/2,
        center_point.y - height/2,
        center_point.x + width/2,
        center_point.y + height/2
    )

def generate_brooklyn_voronoi(buildings_path=BUILDINGS_PATH, parcels_path=PARCELS_PATH, 
                             output_dir=VORONOI_DIR, sample_file=SAMPLE_FILE,
                             buffer_distance=BUFFER_DISTANCE):
    """Generate Voronoi tessellations for the same Brooklyn samples as ground truth."""
    
    # Load datasets and convert to web mercator projection
    buildings_df = gpd.read_file(buildings_path).to_crs(3857)
    parcels_df = gpd.read_file(parcels_path).to_crs(3857)
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Check if sample file exists
    if not os.path.exists(sample_file):
        print(f"Error: Sample file {sample_file} not found.")
        print("Please run the ground truth generator script first.")
        return
    
    # Load sample indices
    sampled_indices = np.load(sample_file)
    print(f"Loaded {len(sampled_indices)} sample indices from {sample_file}")
    
    # Generate citywide Voronoi tessellation if enabled
    citywide_voronoi = None
    if USE_CITYWIDE_VORONOI:
        print("Generating citywide Voronoi tessellation...")
        try:
            # Use convex hull of all parcels as boundary
            city_boundary = parcels_df.unary_union.convex_hull
            
            # Generate the citywide tessellation
            citywide_voronoi = generate_voronoi_tessellation(buildings_df, city_boundary)
            print(f"Generated citywide Voronoi with {len(citywide_voronoi)} cells")
        except Exception as e:
            print(f"Error generating citywide Voronoi: {e}")
            print("Falling back to per-area tessellation.")
            USE_CITYWIDE_VORONOI = False
    
    # Process each sampled parcel
    print(f"Generating {len(sampled_indices)} Voronoi tessellations...")
    for i, idx in enumerate(tqdm_base(sampled_indices, desc="Processing parcels")):
        # Get the specific parcel
        try:
            parcel = parcels_df.loc[idx]
        except:
            print(f"Error accessing parcel at index {idx}")
            continue
        
        # Create area of interest
        if USE_CITYWIDE_VORONOI:
            # Use rectangular view extent with adjusted size to match new zoom level
            view_width = buffer_distance * 1.5  # Adjusted from 2
            view_height = buffer_distance * 1.5  # Adjusted from 2
            view_geometry = create_view_box(parcel.geometry.centroid, view_width, view_height)
            
            # Extract Voronoi cells that intersect with view area
            voronoi_in_view = citywide_voronoi[citywide_voronoi.intersects(view_geometry)].copy()
            
            # Clip the cells to the view boundary
            clipped_cells = []
            for _, cell in voronoi_in_view.iterrows():
                clipped_geom = cell.geometry.intersection(view_geometry)
                if clipped_geom.area > 0:
                    clipped_cells.append({
                        'geometry': clipped_geom,
                        'color': cell['color']
                    })
            
            # Create GeoDataFrame from clipped cells
            voronoi_gdf = gpd.GeoDataFrame(clipped_cells, crs=buildings_df.crs)
        else:
            # Use buffer around parcel - adjusted for better zoom level
            area_geometry = parcel.geometry.buffer(buffer_distance * 0.75)  # Reduced buffer size
            
            # Get buildings within buffer
            buildings_in_area = buildings_df[buildings_df.geometry.within(area_geometry)]
            
            # Generate Voronoi tessellation for this area
            voronoi_gdf = generate_voronoi_tessellation(buildings_in_area, area_geometry)
        
        # Generate the output image if we have Voronoi cells
        if not voronoi_gdf.empty:
            # Define area geometry for rendering (consistent between methods)
            if USE_CITYWIDE_VORONOI:
                render_geometry = view_geometry
            else:
                render_geometry = parcel.geometry.buffer(buffer_distance * 0.75)  # Adjusted to match the buffer
            
            # Generate output filename
            output_filename = f"brooklyn_{i:06d}_voronoi.png"
            output_path = os.path.join(output_dir, output_filename)
            
            # Get buildings in the area for reference if needed
            if USE_CITYWIDE_VORONOI:
                buildings_in_area = buildings_df[buildings_df.geometry.within(view_geometry)]
            else:
                buildings_in_area = buildings_df[buildings_df.geometry.within(render_geometry)]
            
            # Render and save image
            render_voronoi_tessellation(
                voronoi_gdf,
                render_geometry,
                output_path,
                buildings_gdf=buildings_in_area,
                include_buildings=False  # Only Voronoi cells
            )
    
    print(f"Completed generating {len(sampled_indices)} Voronoi tessellations.")
    print(f"Output saved to: {output_dir}")

#########################################################
# MAIN EXECUTION
#########################################################

if __name__ == "__main__":
    # Generate Voronoi tessellations
    generate_brooklyn_voronoi()